In [4]:
using Dates

include("src/macros.jl");
include("src/ops/Collect.jl");
include("src/ops/Aggregate.jl");

In [5]:
data = [
    (DateTime("2020-01-01T12:00:00"), 1.1),
    (DateTime("2020-01-01T12:01:00"), 1.0),
    (DateTime("2020-01-01T12:03:00"), 1.3),
    (DateTime("2020-01-01T12:06:00"), 1.6),
    (DateTime("2020-01-01T12:11:00"), 1.2),
    (DateTime("2020-01-01T13:11:00"), 1.7),
    (DateTime("2020-01-01T14:11:00"), 1.8),
]

collector = Collect{Any}()
agg = @streamops begin
    Aggregate{eltype(data),DateTime}(;
        key_fn=v -> round(v[1], Dates.Minute(5), RoundUp),
        agg_fn=(key, buffer) -> begin
            (key, last(map(x -> x[2], buffer)))
        end
    )
    @filter !isnothing
    collector
end

agg.(data)
collector.out

5-element Vector{Any}:
 (DateTime("2020-01-01T12:00:00"), 1.1)
 (DateTime("2020-01-01T12:05:00"), 1.3)
 (DateTime("2020-01-01T12:10:00"), 1.6)
 (DateTime("2020-01-01T12:15:00"), 1.2)
 (DateTime("2020-01-01T13:15:00"), 1.7)